In [14]:
# -*- coding: utf-8 -*-
# Created in Python 3 using Spyder, Jupyter Notebooks, and Google Colab
# Created by Alexandra Beno
# Computer Science Capstone
# C964
# Student ID 011095454

import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from collections import Counter
import matplotlib.ticker as mticker
from sklearn.metrics import silhouette_score, silhouette_samples
import numpy as np
from ipywidgets import Button, Output, Text
from IPython.display import display, clear_output

# put dataset of Steam Bestselling Games into dataframe. This data is obtained from https://www.kaggle.com/datasets/hbugrae/best-selling-steam-games-of-all-time?resource=download
# the dataset was gathered on June 1st, 2025 and has headers already.
# the user_defined_tags column features a list of tags for each game. on Steam, users can assign any game they wish any tag they like. the lists included in this dataset have been standardized and cleaned into 42 representative genres and tags.
df = pd.read_csv("bestSelling_games.csv")

# here variables are declared to refer to specific columns of the dataframe easier.
gameList = df.iloc[:, 0]    # this variable is a list of the names of the games.
tagList = df.iloc[:, 5]          # this variable is a list of each game's list of tags.

# vectorizer is used to prepare the data by converting the list of tags into a machine-readable form.
# a matrix is prepared for the program to examine data using the game tags, converting them from being lists in single cells.
df['Tags'] = tagList.astype(str)
df['Tag_List'] = df['Tags'].apply(lambda x: [t.strip().lower() for t in x.split(',') if t.strip() != ''])
vectorizer2 = TfidfVectorizer(stop_words='english')
matrix = vectorizer2.fit_transform(tagList)

# in this section k-means clustering is used to group the games into groups based on their tags.
# each game is assigned to a cluster based on the tags it possesses.
k = 15  # a value of 15 was used for k in this program, though analysis of this is an area for future improvement, as it was unclear during development of the program what the best value for k would end up being.
kmeans = KMeans(n_clusters=k)
kmeans.fit(matrix)
similarity = cosine_similarity(matrix, matrix)
df['Cluster'] = kmeans.labels_

# here the clusters' tags are summarized so that the TSNE plot is more human-readable.
clusterTags = {} # an array storing summaries of cluster tags to be used when the clusters are plotted
for i in sorted(df['Cluster'].unique()): # iterate through each cluster
  clusterTagList = [tag for tags in df[df['Cluster'] == i]['Tag_List'] for tag in tags] # get all the tags in the gluster
  clusterTagCount = Counter(clusterTagList) # count the tags
  bestTags = [t for t, _ in clusterTagCount.most_common(5)] # find the 5 most common to use in a plot later
  clusterTags[i] = bestTags # store those 5 tags in the cluster summary array

# in this section, silhouette coefficients are used to determine the accuracy of the k-means clustering above.
# to do this, a sample game is taken from each cluster, and its distance to its own cluster vs the other clusters is determined
# the higher the score, the closer it is to its own cluster and the further from others.
silAvg = silhouette_score(matrix, kmeans.labels_)
#print(f"\nAverage Silhouette Coefficient for k={k}: {silAvg}")
silSample = silhouette_samples(matrix, kmeans.labels_)

# in this section, a silhouette analysis is conducted on the samples taken above.
# their coefficients are found and charted on a silhouette plot
# the silhouette coefficient values are X, and the cluster number the sample is taken from is the y.
# the size, vertically, of the cluster indicates the number of data points in the cluster
# the horizontal length of each sample indicates the silhouette coefficient of the sample.
def silPlot():
  plt.figure(figsize=(10, 6)) # size of plot
  silDown = 10 # starting place
  for i in range(k): # iterate through each cluster
    silCluster = silSample[kmeans.labels_ == i] # current cluster
    silCluster.sort() # sort based on silhoette coefficient
    silSize = silCluster.shape[0] # the size of the cluster
    silUp = silDown + silSize # ending place
    plt.fill_betweenx(np.arange(silDown, silUp), 0, silCluster) # plot the cluster on the graph
    plt.text(-0.05, silDown + 0.5 * silSize, str(i)) # label the plotted cluster
    silDown = silUp + 10 # next starting place
  # below are labels for the silhouette plot
  plt.axvline(x=silAvg, color="red", linestyle="--")
  plt.xlabel("The silhouette coefficient values")
  plt.ylabel("Cluster label")
  plt.title(f"The silhouette plot for k={k} (Average = {silAvg:.3f})")
  plt.show()
# a freaky bug crawled across my wall as i typed this. i generally have a 3 strikes rule when it comes to bugs and other small creatures like that. so i like the creepy bug get away this time. but if he messes with me again i dont know if im going to be able to spare him again cause he was really scary.
# in the spirit of halloween, i think i will let the bug live.

# in this section below, t-distributed stochastic neighbor embedding (TSNE) is used to visualize the clusters.
# this is a method of visualizing the similarity of various datapoints, and is generally used for clustering. here, it lets us visualize the clusters that the k-means algorithm has sorted the games into.
visualiser = TSNE(perplexity=40) # here the main variables for the TSNE function are declared. as most are the default, the only one left is perplexity, which is related to the number of nearest neighbors that is used in the algorithm. i chose 40, as i want the plot to be accurate but i also do not want it to take a long time to appear.
# the data is fit to the grid using the TSNE algoritm
visualiserGrid = visualiser.fit_transform(matrix.toarray())
df['TSNE1'] = visualiserGrid[:, 0]
df['TSNE2'] = visualiserGrid[:, 1]

# here the clusters are actually plotted on a grid for the user to see
def tsnePlot():
  plt.figure(figsize=(10, 7)) # size of the plot
  scatter = plt.scatter(df['TSNE1'], df['TSNE2'], c=df['Cluster'], cmap='tab10', alpha=0.7)
  plt.title('t-SNE Visualization of Game Clusters')
  plt.colorbar(scatter, label='Cluster')
  clusterCenter = df.groupby('Cluster')[['TSNE1', 'TSNE2']].mean() # the center of each cluster
  for i, row in clusterCenter.iterrows(): # iterate through the center of each cluster
    label = ", ".join(clusterTags[i][:3]) # a blank label
    plt.text(row['TSNE1'], row['TSNE2'], label, fontsize=9, weight='bold') # label the center of each cluster with the top 5 tags of that cluster. this is an area for future improvement, to make sure the labels do not overlap
  plt.show()

# this section is the function used to recommend games to the user. the input is a string that matches a game on the list of Steam Best-sellers and the desired number of recommendations, and the output is a list of games that are similar to the inputted game based on tags.
def recommender(game, results): # (game to find similarities to as a string, number of results desired as int)
  if game not in gameList.values: # make sure that the game is actually on the list of best-sellers as of June 1st, 2025. Sorry Silksong!
    print("Game not found.")
    return []
  spot = df[df.iloc[:,0] == game].index[0] # the location of the game in the dataframe
  myCluster = df.loc[spot, 'Cluster'] # the cluster of the game
  clusteredGames = df[df['Cluster'] == myCluster] # the games also in this cluster
  clusteredSpots = clusteredGames.index.tolist() # the locations of the other games in this cluster
  score = [(i, similarity[spot][i]) for i in clusteredSpots if i != spot] # the similarity is computed by comparing the locations of the game and the others in its cluster.
  score = sorted(score, key=lambda x: x[1], reverse=True)[:results] # the results are sorted and cleaned up
  recommend = df.iloc[[i[0] for i in score], 0].values # the most similar games are chosen
  return recommend



# this function is used to display the results and graphs for a specific game recommendation.
def recommendMode(game): # takes a game as a string
  recommendations = recommender(game, 5)
  if len(recommendations) == 0:
    return
  print(f"\nGames similar to {game}:") # prints the list of games similar to the input.
  for i in recommendations:
    print(" -", i)

  # the recommended games are copied out into their own dataframe to be manipulated and cleaned for the purposes of visualization.
  recommendedGames = df[df.iloc[:, 0].isin(recommendations)].copy()

  # in the following section, a bar graph is created to show the user the platforms that the games they are being recommended are available on. (generally PC, sometimes mac or linux)
  platformList = recommendedGames.iloc[:, 6].dropna().apply(lambda x: [p.strip() for p in x.split(',')]) # the list of just the platforms
  platformCount = pd.Series([p for sublist in platformList for p in sublist]).value_counts() # the platforms are totalled
  plt.figure(figsize=(8, 5)) # plot size
  platformCount.plot(kind='bar', color='purple', edgecolor='black') # graph the counts of platforms, with the x axis being the platforms and the y axis being the counts of each platform.
  plt.title(f"Platforms of Recommended Games for {game}")
  plt.xlabel("Platform")
  plt.ylabel("Count")
  plt.xticks(rotation=0)
  plt.show()

  # in the following section, the data in the recommended games list is prepared to ensure it is ready for a plot that displays the reviews of the games.
  recommendedGames['Positive %'] = recommendedGames.iloc[:, 1].astype(float) # the percentage of reviews that are positive, as a float
  recommendedGames['Total Reviews'] = recommendedGames.iloc[:, 2].astype(int) # the total number of reviews, as an int
  recommendedGames['Truncated'] = recommendedGames.iloc[:, 0].apply(lambda x: x[:20] + '…' if len(x) > 20 else x) # the title of the game, truncated to 20 characters for better display.


  # now the total number of reviews is plotted as the left y-axis and the percentage of positive reviews is plotted as the right y axis. the list of games is the x-axis.
  fig, left = plt.subplots(figsize=(10, 5)) # size of plot
  left.bar(recommendedGames['Truncated'], recommendedGames['Total Reviews'], color='purple', label='Total Reviews') # total reviews on left, game names on bottom
  left.set_xlabel("Game")
  left.set_ylabel("Total Reviews", color='purple')
  left.tick_params(axis='x', rotation=45)
  for label in left.get_xticklabels(): # iterate through the list of x-axis labels
    label.set_ha('right') # align the label
  left.yaxis.set_major_formatter(mticker.StrMethodFormatter('{x:,.0f}'))
  right = left.twinx()
  right.plot(recommendedGames['Truncated'], recommendedGames['Positive %'], color='darkorange', marker='o', label='Positive %') # percentage positive on right
  right.set_ylabel("Positive Reviews (%)", color='darkorange')
  plt.title(f"Review Summary for Recommended Games Similar to {game}")
  plt.show()

# the section below sets up the user interface and the buttons and text fields that the user will use to navigate.
out = Output()

# this function calls the functions to draw the t-SNE and silhouette coefficient visualizations of the clusters when the "Data Analysis" button is pressed.
def dataClick(b):
  with out:
    clear_output()
    tsnePlot()
    silPlot()

# this function asks the user for a game title when the "Recommendations" button is pressed, and uses that to recommend games when the "Confirm" button is pressed.
def recommendClick(b):
  with out:
    clear_output()
    entry = Text(placeholder = "Enter a game title")
    # this function takes the value from the entry field and procures recommendations when the "Confirm" button is pressed.
    def confirmClick(b):
      with out:
        clear_output()
        inputGame = entry.value.strip()
        if not inputGame:
          print("Please enter a game title.")
        recommendMode(inputGame)
    # setting up the "Confirm" button
    confirmBtn = Button(description = "Confirm")
    confirmBtn.on_click(confirmClick)

    display(entry, confirmBtn) # display the entry field and the "Confirm" button

# setting up the "Recommendations" button
recommendBtn = Button(description = "Recommendations")
recommendBtn.on_click(recommendClick)
# setting up the "Data Analysis" button
dataBtn = Button(description = "Data Analysis")
dataBtn.on_click(dataClick)

display(recommendBtn, dataBtn, out) # display the "Recommendations" and "Data Analysis" buttons when the program starts.



Button(description='Recommendations', style=ButtonStyle())

Button(description='Data Analysis', style=ButtonStyle())

Output()